# `wavinfo` Demonstration

The `wavinfo` module allows you to read most of the metadata formats that are available for WAV files.

## Opening a WAV file for reading metadata

The entry point for wavinfo is the `WavInfoReader` class:

In [3]:
from wavinfo import WavInfoReader

path = '../tests/test_files/sounddevices/A101_1.WAV'

info = WavInfoReader(path)

Once you have a `WavInfoReader`, you can access different metadata systems or "scopes."

The scopes that are presently supported are: `fmt`, `data`, `ixml`, `bext`, `info`, `adm`, `cues`, and `dolby`. Each of these is an attribute of a `WavInfoReader` object.

Each scope roughly corresponds to a vendor-defined metadata system. Many scopes directly represent a specific file *chunk*, like `fmt` or `ixml`, and some may involve data read from many chunks. Examples of this would include `cues` or `adm`.


## Metadata Scopes

### `data` and `fmt`: Basic WAV Data

The length of the file in frames (interleaved samples) and bytes is available, as is the contents of the format chunk.

In [4]:
(info.data.frame_count, info.data.byte_count)

(240239, 1441434)

The `fmt` scope allows the client to read metadata from the WAVE format description.

In [5]:
(info.fmt.sample_rate, info.fmt.channel_count, info.fmt.block_align, info.fmt.bits_per_sample)

(48000, 2, 6, 24)

### `bext`: Broadcast WAV Extension

The `bext` scope allows the client to access Broadcast-WAV metadata. 

In [6]:
print(info.bext.description)
print("----------")
print("Originator:", info.bext.originator)
print("Originator Ref:", info.bext.originator_ref)
print("Originator Date:", info.bext.originator_date)
print("Originator Time:", info.bext.originator_time)
print("Time Reference:", info.bext.time_reference)
print(info.bext.coding_history)

sSPEED=023.976-ND
sTAKE=1
sUBITS=$12311801
sSWVER=2.67
sPROJECT=BMH
sSCENE=A101
sFILENAME=A101_1.WAV
sTAPE=18Y12M31
sTRK1=MKH516 A
sTRK2=Boom
sNOTE=

----------
Originator: Sound Dev: 702T S#GR1112089007
Originator Ref: USSDVGR1112089007124001008206301
Originator Date: 2018-12-31
Originator Time: 12:40:00
Time Reference: 2190940753
A=PCM,F=48000,W=24,M=stereo,R=48000,T=2 Ch



### `ixml`: iXML Production Recorder Metadata

In [7]:
print("iXML Project:", info.ixml.project)
print("iXML Scene:", info.ixml.scene)
print("iXML Take:", info.ixml.take)
print("iXML Tape:", info.ixml.tape)
print("iXML File Family Name:", info.ixml.family_name)
print("iXML File Family UID:", info.ixml.family_uid)

iXML Project: BMH
iXML Scene: A101
iXML Take: 1
iXML Tape: 18Y12M31
iXML File Family Name: None
iXML File Family UID: USSDVGR1112089007124001008206300


### `cues`: Cues Metadata

Cue time markers are accessible through the `cues` scope. The `each_cue` method returns an iterator that yields a tuple of each cue "name" or integer UID, and sample location. 

In [9]:
path = "../tests/test_files/cue_chunks/STE-000.wav"
info = WavInfoReader(path)

for cue in info.cues.each_cue():
    print(f"Cue ID: {cue[0]}")
    print(f"Cue Offset: {cue[1]}")

Cue ID: 1
Cue Offset: 29616
Cue ID: 2
Cue Offset: 74592
Cue ID: 3
Cue Offset: 121200


There are also convenience methods to get the appropriate label and note for a given marker. (Note here also `WavInfoReader`'s facility for overriding default text encodings.)

In [18]:
path = "../tests/test_files/cue_chunks/izotoperx_cues_test.wav"
info = WavInfoReader(path, info_encoding="utf-8")

for cue in info.cues.each_cue():
    print(f"Cue ID: {cue[0]}")
    label, note = info.cues.label_and_note(cue[0])
    print(f"    Label: {label}")
    print(f"    Note: {note or '<NO NOTE>'}")

Cue ID: 1
    Label: Marker 1
    Note: <NO NOTE>
Cue ID: 2
    Label: Marker 2
    Note: Marker Comment 1
Cue ID: 3
    Label: Marker 3
    Note: Лорем ипсум долор сит амет, тимеам вивендум хас ет, цу адолесценс дефинитионес еам.
